In [1]:
%load_ext autoreload

In [4]:
%autoreload

import os
import sys
import numpy as np
sys.path.append('./src-py')
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from prepare_dataset import *
import datasets

### Preparing the Data:

In [34]:
def process_dataset_for_training(tokenizer, dataset_path, output_path, max_num_turns=40):
    conv_dataset = datasets.load_from_disk(dataset_path)
    conv_dataset = datasets.Dataset.train_test_split(conv_dataset, test_size=0.1, seed=123)

    train_journalist_dataset = prepare_dataset(tokenizer, conv_dataset['train'], role="Journalist", max_num_turns=max_num_turns)
    train_researcher_dataset = prepare_dataset(tokenizer, conv_dataset['train'], role="Researcher", max_num_turns=max_num_turns)
    
    test_journalist_dataset = prepare_dataset(tokenizer, conv_dataset['test'], role="Journalist", max_num_turns=max_num_turns)
    test_researcher_dataset = prepare_dataset(tokenizer, conv_dataset['test'], role="Researcher", max_num_turns=max_num_turns)

    train_journalist_dataset = split_dataset_by_clm(train_journalist_dataset, 'paper_id')
    train_researcher_dataset = split_dataset_by_clm(train_researcher_dataset, 'paper_id')

    train_journalist_dataset.save_to_disk(output_path + '/train_journalist_ds')
    train_researcher_dataset.save_to_disk(output_path + '/train_researcher_ds')
    
    test_journalist_dataset.save_to_disk(output_path + '/test_journalist_ds')
    test_researcher_dataset.save_to_disk(output_path + '/test_researcher_ds')

In [ ]:
# Prepare data from  LLAMA-3
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
tokenizer.pad_token = tokenizer.eos_token  # for older tokenizers
process_dataset_for_training(tokenizer, 
                             '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds-cleaned/', 
                             '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds-preprocessed-for-llama3/')

In [ ]:
# Prepare data from  Qwen
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
tokenizer.pad_token = tokenizer.eos_token  # for older tokenizers
process_dataset_for_training(tokenizer, 
                             '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds-cleaned/', 
                             '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds-preprocessed-for-qwen/',
                            max_num_turns=20)

 12%|████████▌                                                                 | 1540/13294 [00:11<01:28, 133.44it/s]

-----------

In [38]:
train_researcher_dataset = datasets.load_from_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds-preprocessed-for-qwen/train_researcher_ds')

In [39]:
train_researcher_dataset

DatasetDict({
    train: Dataset({
        features: ['paper_id', 'paper_title', 'paper_text', 'prompt', 'completion'],
        num_rows: 76278
    })
    test: Dataset({
        features: ['paper_id', 'paper_title', 'paper_text', 'prompt', 'completion'],
        num_rows: 8492
    })
})

In [10]:
print(train_researcher_dataset['train']['prompt'][1])
print(train_researcher_dataset['train']['completion'][1])

[{'content': 'You are a helpful and expert researcher answering questions about your scientific paper.', 'role': 'system'}, {'content': '[PAPERT-TITLE]\nBlood flow–sensing protein protects against atherosclerosis in mice\n[PAPER]\nAbstract Endothelial cells transduce mechanical forces from blood flow into intracellular signals required for vascular homeostasis. Here we show that endothelial NOTCH1 is responsive to shear stress, and is necessary for the maintenance of junctional integrity, cell elongation, and suppression of proliferation, phenotypes induced by laminar shear stress. NOTCH1 receptor localizes downstream of flow and canonical NOTCH signaling scales with the magnitude of fluid shear stress. Reduction of NOTCH1 destabilizes cellular junctions and triggers endothelial proliferation. NOTCH1 suppression results in changes in expression of genes involved in the regulation of intracellular calcium and proliferation, and preventing the increase of calcium signaling rescues the ce